# Logistic Regression with Scikit Learn

*Adapted from https://github.com/justmarkham*

### Libraries

- [scikit-learn](http://scikit-learn.org/stable/)
- pandas
- matplotlib

In this tutorial we will see some basic examples of Logistic Regression for classification.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import cross_val_score
import seaborn as sns
%matplotlib inline

## Classification with Logistic Regression

|*|continuous|categorical|
|---|---|---|
|**supervised**|regression|**classification**|
|**unsupervised**|dim. reduction|clustering|

# Predicting Titanic survival with Logistic Regression

Let's use the data obtained by the _Encyclopedia Titanica_ to predict if a passenger survived the Titanic disaster.

<img src="img/titanic.jpg" width="600">

Let's import the dataset _titanic.csv_ (_hint_ use the `read_csv` pandas function):

In [2]:
titanic = pd.read_csv('data/titanic.csv')
titanic.head(5)

,survived,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked
0,1,"Allen, Miss. Elisabeth Walton",female,29.0000,0,0,24160,211.3375,B5,S
1,1,"Allison, Master. Hudson Trevor",male,0.9167,1,2,113781,151.5500,C22 C26,S
2,0,"Allison, Miss. Helen Loraine",female,2.0000,1,2,113781,151.5500,C22 C26,S
3,0,"Allison, Mr. Hudson Joshua Creighton",male,30.0000,1,2,113781,151.5500,C22 C26,S
4,0,"Allison, Mrs. Hudson J C (Bessie Waldo Daniels)",female,25.0000,1,2,113781,151.5500,C22 C26,S


What are the **features**?
- name: Name of the passenger
- sex: Male or Female
- age: Age in years
- sibsp: # of siblings / spouses aboard the Titanic
- parch: # of parents / children aboard the Titanic
- ticket: Ticket number
- fare: Ticket price
- cabin: Cabin number
- embarked: Port of Embarkation

What is the **response**?
- survived: whether the passenger survived the disaster or not

Print the number of survivors and death passengers, as well as the percentage of survivors. Is the dataset balanced?

In [3]:
dead = titanic[titanic['survived']==0]
survived = titanic[titanic['survived']==1]

print("Survived {0}, Dead {1}".format(len(survived), len(dead)))
print("Survived {:.2%}".format(len(survived)/len(titanic)))

Survived 500, Dead 809
Survived 38.20%


Specify the columns to use as features:

In [4]:
titanic_features = ['sex', 'age', 'sibsp', 'parch', 'fare']

For the sake of this execise, we can assume the other features (name, cabin number, embarked) are not predictive.

### Let's prepare the feature vector for the training

The dataset contains one categorical variable: sex (male|female)

We need to convert it to a numerical variable. Use the pandas method `get_dummies` to take care of this. Check https://pandas.pydata.org/docs/reference/api/pandas.get_dummies.html

In [5]:
# The features vector
X = pd.get_dummies(titanic[titanic_features])
X.head()

,age,sibsp,parch,fare,sex_female,sex_male
0,29.0000,0,0,211.3375,1,0
1,0.9167,1,2,151.5500,0,1
2,2.0000,1,2,151.5500,1,0
3,30.0000,1,2,151.5500,0,1
4,25.0000,1,2,151.5500,1,0


The categorical feature _sex_ is converted in 2 boolean features.

Titanic sank in 1912: it was a lot of time ago! Some data may be missing. Let's check if there are undefined values. Use pandas' `isna` for this purpose: https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.isna.html

In [6]:
len(X[X.isna().any(axis=1)])

264

Let's try to fix the data with a basic imputation method: replacing the missing values with the mean. Use pandas' `fillna` for this purpose: https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.fillna.html

More info: https://en.wikipedia.org/wiki/Imputation_(statistics)

In [7]:
X = X.fillna(X.mean())

len(X[X.isna().any(axis=1)])

0

Create the label vector `y`:

In [8]:
y = titanic['survived']

Let's create a Logistic Regression model...

In [9]:
logistic = LogisticRegression(solver='lbfgs')

... and evaluate the precison/recall with a cross validation (10 splits). For this, use the `cross_val_score` implementation provided by `sklearn` and already imported above. _Hint:_ check the `scoring` argument of this function: https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_val_score.html

In [10]:
precision = cross_val_score(logistic, X, y, cv=10, scoring="precision")
recall = cross_val_score(logistic, X, y, cv=10, scoring="recall")

# Precision: avoid false positives
print("Precision: %0.2f (+/- %0.2f)" % (precision.mean(), precision.std() * 2))
# Recall: avoid false negatives
print("Recall: %0.2f (+/- %0.2f)" % (recall.mean(), recall.std() * 2))

Precision: 0.72 (+/- 0.13)
Recall: 0.68 (+/- 0.16)


### Explore the model output

Let's create a new Logistic Regression model and train it on the full dataset:

In [11]:
logistic = LogisticRegression(solver='lbfgs')
logistic.fit(X.values, y)

LogisticRegression()

Of course, since we trained the whole dataset, we don't have new samples to predict, but we can predict the outcome and the relative probability for some artificial samples. Would you have survived?

Remember the features:

In [12]:
X.columns

Index(['age', 'sibsp', 'parch', 'fare', 'sex_female', 'sex_male'], dtype='object')

Would a man, 25 years old without relative onboard, and with a fare of 100 survive? _Hint:_ use pandas' `predict` to make the prediction.

In [13]:
test = [25, 0, 0, 100, 0, 1]
"YES" if logistic.predict([test])[0] > 0 else "NO"

'NO'

What is the probability distribution behind this prediction? _Hint:_ use pandas' `predict_proba` to find the prediction distribution.

In [14]:
logistic.predict_proba([test])

array([[0.5528599, 0.4471401]])

What about a woman, 35 years old, alone onboard and with the same fare?

In [15]:
test = [35, 0, 0, 100, 1, 0]

print("YES" if logistic.predict([test])[0] > 0 else "NO")
print(logistic.predict_proba([test])[0])

YES
[0.11464109 0.88535891]
